# Getting Started

This notebook provides a quick introduction to using the IBM Envizi Emissions API for calculating greenhouse gas emissions. We'll walk through the two kinds of sample json requests that the APIs accept and learn how to select the appropriate format based on specific use case requirements.

## POST

### By Factor Type

IBM Envizi Emissions API uses a JSON payload structure that follows the "What, Where, When" pattern:

- **What** (activity): what type of activity generated emissions (e.g., electricity consumption, fuel combustion)
- **Where** (location): where the activity took place (country, region, etc.)
- **When** (time): when the activity occurred (date, time period)

Let's create a simple payload for calculating emissions from electricity consumption:

In [ ]:
{
  "activity": {
    "type": "electricity",
    "value": 10,
    "unit": "kWh"
  },
  "location": {
    "country": "usa",
    "stateProvince": "new york"
  },
  "time": {
    "date": "2025-01-04"
  },
  "includeDetails": true
}

#### Parameters

1. **Activity (What)**:
   - `type`: The type of activity (electricity consumption)
   - `value`: The amount of electricity consumed (10 kilowatt-hours)
   - `unit`: The unit of measurement (kWh)

2. **Location (Where)**:
   - `country`: The country where the electricity was consumed, a mandatory parameter (USA)
   - `stateProvince` is accepted as an optional parameter (new york)
   - Either `stateProvince` or `powerGrid` can be supplied as an optional input in case of calculating emissions on Location API.
   - Location is crucial as emissions factors can vary significantly by region

3. **Time (When)**:
   - `date`: The date when the electricity was consumed (January 4, 2025)
   - This helps the API select the appropriate emissions factors for that time period

4. **includeDetails**:
   - A boolean flag that determines whether the API response includes detailed calculation information

### By Factor ID

The most significant difference in this payload is the use of `factorId` instead of the location, time and activity type. 

This approach:

- Directly references a specific emissions factor in the IBM Envizi database
- Bypasses the need to specify location, time, and activity type
- Provides a deterministic way to ensure the exact same factor is used consistently

Let's create a simple payload for fetching emissions associated with a unique factorID along with the unit and value parameters:

In [ ]:
{
  "activity": {
    "factorId": 29902,
    "unit": "gal",
    "value": 0.9
  },
  "includeDetails": true
}

The main reason is to override the Envizi Factor Selection Algorithm (FSA) to deterministically use a factor in a calculation. 

This approach is valuable when you need:

1. **Regulatory Compliance**: Ensure specific approved factors are used for regulatory reporting
2. **Temporal Consistency**: Maintain the same factor across multiple reporting periods
3. **Clear Audit Trail**: Document exactly which factors were used in calculations
4. **Calculation Reproducibility**: Guarantee identical calculation results over time, regardless of factor database updates

## GET

All endpoints allow for GET calls in addition to POST, where the name of the parameters in the POST payload are represented instead by parameters declared on the url path.

`https://api.ibm.com/ghgemissions/run/v3/carbon/location?type=electricity&value=10&unit=kwh&country=usa&stateProvince=new%20york&date=2025-01-04&includeDetails=true`

## includeDetails

When `includeDetails` is set to `true`, the response includes critical information for audit purposes:

1. **Complete Factor Set Information**: This provides auditors with:
- The exact source of emissions factors
- Official documentation links for verification
- Information about the data provider's authority
- Licensing information confirming legitimate use

2. **Temporal Relevance and Version Control**: Auditors can verify:
- The emissions factors are appropriate for the reporting period 
- The factors are the most current available 
- The data is from an authoritative source 
- When the factors will be reviewed next 

3. **Precise Factor Values with Geographic Specificity**: This provides:
- The exact numerical values used for each greenhouse gas component
- Geographic specificity (United States)
- Unique factor identifiers (factorId) for traceability

4. **Unit Conversion Transparency**: This confirms:
- The input unit matches the factor unit
- If any conversion was necessary, and the conversion ratio used

```
{
  "result": {
    "transactionId": "b8ec6b22-46b5-4107-9597-348079157127",
    "totalCO2e": 2.2284403999999998,
    "CO2": 2.2212781,
    "CH4": 0.0035562,
    "N2O": 0.0036061,
    "indirectCO2e": 0.1197581,
    "unit": "kgCO2e",
    "description": "The electricity emissions factor used to calculate this result was obtained from the  year 2022 Managed - eGRID & US Climate Leaders factor set for the area United States and the region New York."
  },
  "data": {
    "unit": [
      {
        "name": "CH4(kgCO2e/unit)"
      },
      {
        "name": "CO2(kgCO2e/unit)"
      },
      {
        "name": "Indirect CO2e(kgCO2e/unit)"
      },
      {
        "name": "N2O(kgCO2e/unit)"
      },
      {
        "name": "Total CO2e(kgCO2e/unit)",
        "description": "It equals the sum of CO2, CH4 and N2O per unit factors expressed in kg of CO2-equivalents. "
      }
    ],
    "area": [
      {
        "internalName": "New York",
        "standardName": "New York",
        "name": "New York",
        "stdCode": "ISO 3166-2:US-NY"
      }
    ],
    "areaType": [
      {
        "name": "State"
      }
    ],
    "factorDataTypeCategories": [
      {
        "name": "Other Metrics"
      }
    ],
    "factorSet": [
      {
        "type": "Managed - eGRID & US Climate Leaders",
        "description": "The eGRID & US Climate Leaders factor set combines data from three U.S. EPA sources: eGRID, US Climate Leaders, and USEEIO.\n\neGRID provides emissions, generation, and fuel mix data for nearly all U.S. electricity generation. It supports carbon footprinting, GHG inventories, and avoided emissions analysis.\n\nUS Climate Leaders offers GHG emission factors for fuels, refrigerants, waste, business travel, and distribution, updated every 1–2 years.\n\nUSEEIO includes spend-based emission factors for 394 commodities across 66 categories, supporting supply chain emissions analysis.\n\nTogether, these datasets support robust, standardized emissions modeling across energy use and procurement.",
        "provider": "Federal Government of the United States: Environmental Protection Agency",
        "sourceUrl": "https://www.epa.gov/egrid, https://www.epa.gov/climateleadership/ghg-emission-factors-hub, https://cfpub.epa.gov/si/si_public_record_Report.cfm?dirEntryId=349324&Lab=CESER",
        "displayName": "eGRID & US Climate Leaders",
        "license": "Public",
        "publish": true,
        "calculationOnly": false,
        "descriptionShort": "The eGRID & US Climate Leaders factor set is a combination of the eGRID, US Climate Leaders and USEEIO data sets from the Federal Government of the United States- Environmental Protection Agency.",
        "citation": "USEEIO\nIngwersen, W. AND M. Li. Supply Chain Greenhouse Gas Emission Factors for US Industries and Commodities. U.S. Environmental Protection Agency, Washington, DC, EPA/600/R-20/001, 2020."
      }
    ],
    "factorDataTypeCode": [
      {
        "unitOfMeasure": "kWh"
      }
    ],
    "factorSetVersion": [
      {
        "effectiveFrom": "2022-01-01",
        "publishedFrom": "2024-01-01",
        "reviewOn": "2025-02-01",
        "year": 2022,
        "factorSource": "Year 2022 Data: 1/30/2024.Original file name: eGrid2022_summary_tables.xlsx, tab name: Table 3.GWP source: 2014 IPCC AR5 (N2O GWP = 265, CH4 GWP = 28); Source: https://www.epa.gov/egrid; Grid Gross Loss % taken from US average value in tab name: Table 1."
      }
    ],
    "factorValue": [
      {
        "country": "United States",
        "region": "New York",
        "unit": {
          "name": "N2O(kgCO2e/unit)"
        },
        "value": 3.6061E-4,
        "factorId": 158989
      },
      {
        "country": "United States",
        "region": "New York",
        "unit": {
          "name": "Total CO2e(kgCO2e/unit)",
          "description": "It equals the sum of CO2, CH4 and N2O per unit factors expressed in kg of CO2-equivalents. "
        },
        "value": 0.22284404,
        "factorId": 158989
      },
      {
        "country": "United States",
        "region": "New York",
        "unit": {
          "name": "CO2(kgCO2e/unit)"
        },
        "value": 0.22212781,
        "factorId": 158989
      },
      {
        "country": "United States",
        "region": "New York",
        "unit": {
          "name": "CH4(kgCO2e/unit)"
        },
        "value": 3.5562E-4,
        "factorId": 158989
      },
      {
        "country": "United States",
        "region": "New York",
        "unit": {
          "name": "Indirect CO2e(kgCO2e/unit)"
        },
        "value": 0.01197581,
        "factorId": 158989
      }
    ],
    "factorMetaData": [
      {
        "key": "description",
        "value": "2022 New York State CO2 output emission rate (489.708 lb/MWh), CH4 output emission rate (0.028 lb/MWh), N2O output emission rate (0.003 lb/MWh) converted to kg CO2e/kWh using 1 lb = 0.45359237 kg; Indirect CO2e is calculated as Total CO2e * Grid Gross Loss % / (1 - Grid Gross Loss %), with factors associated with eGRID regions using eGRID region specific GGL values, and factors associated with US state regions using the US average GGL value.",
        "factorId": 158989
      },
      {
        "key": "name",
        "value": "eGRID Electricity Year 2022",
        "factorId": 158989
      }
    ],
    "unitConversion": [
      {
        "inputUnit": "kwh",
        "unitFromDb": "kWh",
        "conversionRatio": "1:1"
      }
    ]
  }
}
```